In [0]:
!pip install scikit-optimize

In [0]:
#imports:
import pandas as pd
import numpy as np
import math
import os
from time import time
import json 


import xgboost


from skopt import BayesSearchCV
from skopt.space import Real, Space, Integer

from matplotlib import pyplot as plt

In [0]:
from google.colab import drive 
drive.mount('/mntDrive') 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [0]:
PREP_DATA_PATH = "/mntDrive/My Drive/GrBoost/Практика/data/input/prepared_facebook_data.csv"
data=pd.read_csv(PREP_DATA_PATH)

RES_DATA_DIR = "/mntDrive/My Drive/GrBoost/Практика/data/results/test 3"
RES_SAVE_DIR = "/mntDrive/My Drive/GrBoost/Практика/data/results/test 3"

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
files = os.listdir(RES_DATA_DIR)
files = [x for x in files if x.find("group_bayes_search")!=-1]

In [0]:
def group_best_params_with_stop(df, stop_iter_num, left_iter_num, right_iter_num):
    best_params = {}
    stop_iter_nums = {}
    best_score = {}

    tmp_df = df.loc[(df['iter_num'] >= left_iter_num) & (df['iter_num'] <= right_iter_num)].copy() #take group
    for i in tmp_df.run_number.unique(): 
      tmp_df_i = tmp_df.loc[tmp_df.run_number==i] #take run
      #cut iterations:
      tmp_df_i_copy = tmp_df_i.copy()
      tmp_df_i_copy.loc[:, "mean_test_score"] = tmp_df_i.loc[:, "mean_test_score"].cummax()      

      try:
          curr_max=tmp_df_i_copy.groupby("mean_test_score").count().reset_index().sort_values(by = "mean_test_score")
          max_score = curr_max.loc[curr_max["mean_test_score"]>=stop_iter_num].index[0] #get first interval with iter_amount more than stop_iter_num
      except:
          max_score = tmp_df_i_copy.mean_test_score.max()
      
      tmp_stop = tmp_df_i.loc[tmp_df_i["mean_test_score"]==max_score].sort_values(by="iter_num").iloc[0]#get first element with best_score

      best_params[i] = tmp_stop['params']
      #best_score[i] = tmp_stop['mean_test_score']

      last_iter_num = tmp_stop['iter_num'] + stop_iter_num - 1
      if last_iter_num > right_iter_num:
          last_iter_num = right_iter_num

      stop_iter_nums[i] = last_iter_num
    return best_params, stop_iter_nums#, best_score       
   

In [0]:
group_df = pd.DataFrame([])
for file_res in files:
    df = pd.read_csv(RES_DATA_DIR+"/"+file_res)    
    df['iter_num'] = range(1, df.shape[0]+1)
    group_df = pd.concat([group_df, df], sort = False)

In [0]:
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

param_dist_group_2 = {'reg_lambda': Real(0.0, 1.0, 'uniform'),
                      'reg_alpha': Real(0.0, 1.0, 'uniform')
                      }
                      
param_dist_group_3 = {'subsample': Real(0.5, 0.9, 'uniform'),
                      'colsample_bytree': Real(0.5, 0.9, 'uniform')
                      } 

In [0]:
stop_iter_num = 100

best_params_first_group, first_stop_iter_nums = group_best_params_with_stop(group_df, 
                                                                            stop_iter_num = stop_iter_num,
                                                                            left_iter_num=1, 
                                                                            right_iter_num=243)

In [0]:
#for params_set_num in best_params_first_group:
for params_set_num in range(1,11):
  clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)
  clf.set_params(**json.loads(best_params_first_group[params_set_num].replace("'", "\"")))
  n_iter_search_sec = int(round((729 - first_stop_iter_nums[params_set_num])/2))
  #second group:
  bayes_search = BayesSearchCV(estimator = clf, 
                                search_spaces = param_dist_group_2,  
                                scoring = 'r2',
                                cv = 5,
                                n_iter = n_iter_search_sec,
                                iid=False, 
                                optimizer_kwargs={
                                    "acq_func": "EI"
                                    }
                                )

  bayes_search.fit(X, y)

  res_rand_2=pd.DataFrame(bayes_search.cv_results_)
  res_rand_2['iter_num'] = range(first_stop_iter_nums[params_set_num]+1, first_stop_iter_nums[params_set_num]+n_iter_search_sec+1)
  res_rand_2['experiment_name']='bayes search for 2 group' 
  res_rand_2['run_number']=params_set_num

  #fit last suitable params
  best_params_second_group, sec_stop_iter_nums = group_best_params_with_stop(res_rand_2, 
                                                         stop_iter_num = stop_iter_num, 
                                                         left_iter_num=first_stop_iter_nums[params_set_num]+1, 
                                                         right_iter_num=n_iter_search_sec)
  clf.set_params(**best_params_second_group[params_set_num])
  print('done for second group')

  #third group:
  n_iter_search_third = 729 - sec_stop_iter_nums[params_set_num]
  bayes_search = BayesSearchCV(estimator = clf, 
                                search_spaces = param_dist_group_3,  
                                scoring = 'r2',
                                cv = 5,
                                n_iter = n_iter_search_third,
                                iid=False, 
                                optimizer_kwargs={
                                    "acq_func": "EI"
                                    }
                                )

  bayes_search.fit(X, y)

  res_rand_3=pd.DataFrame(bayes_search.cv_results_)
  res_rand_3['iter_num'] = range(sec_stop_iter_nums[params_set_num]+1, 729+1)
  res_rand_3['experiment_name']='bayes search for 3 group' 
  res_rand_3['run_number']=params_set_num

  best_params_third_group, third_stop_iter_nums = group_best_params_with_stop(res_rand_3, 
                                                         stop_iter_num = stop_iter_num, 
                                                         left_iter_num=sec_stop_iter_nums[params_set_num]+1, 
                                                         right_iter_num=729)
  res_rand_3=res_rand_3.loc[(res_rand_3['run_number']==params_set_num) & \
                          (res_rand_3['iter_num']<=int(third_stop_iter_nums[params_set_num]))]

  res=pd.concat([res_rand_2,res_rand_3], sort=False)

  sec_idxs = res.loc[res['experiment_name']=='bayes search for 2 group','iter_num'].values
  third_idxs = res.loc[res['experiment_name']=='bayes search for 3 group','iter_num'].values
  intersection = np.intersect1d(third_idxs, sec_idxs)

  res = res.loc[~((res['experiment_name']=='bayes search for 2 group') & (res['iter_num'].isin(intersection)))]

  res.to_csv(RES_SAVE_DIR+'/GR_BAYES_2_and_3_groups_with_stop_'+str(stop_iter_num)+'_'+str(params_set_num), index=False)
  print('done for third group')
  print(res.shape)

done for second group
done for third group
done for second group
done for third group
done for second group
done for third group


In [0]:
"""
files = os.listdir(RES_SAVE_DIR)
files = [x for x in files if x.find("GR_S_2_and_3_groups_with_")!=-1]

stop_iter_num = 100
sec_stop_iter_nums={}
for file_name in files:
  df = pd.read_csv(RES_SAVE_DIR+'/fail_11/'+file_name)

  sec_idxs = df.loc[df['experiment_name']=='random search for 2 group','iter_num'].values
  third_idxs = df.loc[df['experiment_name']=='random search for 3 group','iter_num'].values
  intersection = np.intersect1d(third_idxs, sec_idxs)

  df = df.loc[~((df['experiment_name']=='random search for 2 group') & (df['iter_num'].isin(intersection)))]

  params_set_num = np.unique(df['run_number'])[0]
  
  sec_stop_iter_nums[params_set_num]=np.max(df.loc[df['experiment_name']=='random search for 2 group','iter_num'])

  res_rand_3 = df.loc[df['experiment_name']=='random search for 3 group']

  best_params_third_group, third_stop_iter_nums = group_best_params_with_stop(res_rand_3, 
                                                         stop_iter_num = stop_iter_num, 
                                                         left_iter_num=sec_stop_iter_nums[params_set_num]+1, 
                                                         right_iter_num=729)
  res_rand_3=res_rand_3.loc[(res_rand_3['run_number']==params_set_num) & \
                          (res_rand_3['iter_num']<=int(third_stop_iter_nums[params_set_num]))]
  res_rand_2 = df.loc[df['experiment_name']=='random search for 2 group']
  res=pd.concat([res_rand_2,res_rand_3], sort=False)
  res.to_csv(RES_SAVE_DIR+'/GR_S_2_and_3_groups_with_'+str(stop_iter_num)+'_'+str(params_set_num), index=False)


  print(df.shape)
"""

'\nfiles = os.listdir(RES_SAVE_DIR)\nfiles = [x for x in files if x.find("GR_S_2_and_3_groups_with_")!=-1]\n\nstop_iter_num = 100\nsec_stop_iter_nums={}\nfor file_name in files:\n  df = pd.read_csv(RES_SAVE_DIR+\'/fail_11/\'+file_name)\n\n  sec_idxs = df.loc[df[\'experiment_name\']==\'random search for 2 group\',\'iter_num\'].values\n  third_idxs = df.loc[df[\'experiment_name\']==\'random search for 3 group\',\'iter_num\'].values\n  intersection = np.intersect1d(third_idxs, sec_idxs)\n\n  df = df.loc[~((df[\'experiment_name\']==\'random search for 2 group\') & (df[\'iter_num\'].isin(intersection)))]\n\n  params_set_num = np.unique(df[\'run_number\'])[0]\n  \n  sec_stop_iter_nums[params_set_num]=np.max(df.loc[df[\'experiment_name\']==\'random search for 2 group\',\'iter_num\'])\n\n  res_rand_3 = df.loc[df[\'experiment_name\']==\'random search for 3 group\']\n\n  best_params_third_group, third_stop_iter_nums = group_best_params_with_stop(res_rand_3, \n                                    